In [ ]:
import requests
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image as Image
import os
import glob
from elasticsearch import Elasticsearch
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
images= glob.glob("test/*.jpg")

In [ ]:
num_classes = 37
IMAGE_SHAPE = (256, 256)

m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/resnet_v2_152/feature_vector/4",
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
m.build([None, 256, 256, 3])  # Batch input shape.

In [ ]:
image = Image.open(images[1])
image = np.asarray(image)
image = np.array(image)/255.0
plt.imshow(image)
plt.axis('off')

print(images[1])
print(image.shape)
plt.show()

In [ ]:
vector = m.predict(image[np.newaxis, ...])

In [ ]:
es = Elasticsearch("localhost:9200")
def search(vector):
    index = "document7"

    body = {
        "query" : {
            "script_score": {
                "query": {"match_all": {}},
                "script": {
                    "source": "cosineSimilarity(params.query_vector, doc['image_vector']) + 1.0",
                    "params": {"query_vector": vector}
                }
            }
        }
    }
    
    return es.search(index=index, body=body)

In [ ]:
search(vector[0].tolist())